In [1]:
from importlib import reload
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import os

# print(os.getenv("SEVERA_CLIENT_SCOPE"))

import pandas as pd
import altair as alt
import arrow

import src.severa.base_client
import src.severa.fetch
import src.severa.client
import src.severa.process
import src.database
from src.daterange import DateRange
import src.stable_hash
import datetime

reload(src.severa.base_client)
reload(src.severa.fetch)
reload(src.severa.client)
reload(src.severa.process)
reload(src.database)
reload(src.stable_hash)

span = DateRange(540)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [8]:
t = arrow.utcnow()
span = DateRange(t.shift(months=-4).floor("month"), t.shift(months=2).ceil("month"))

span_past, span_future = span.cut(arrow.utcnow())

async with src.severa.client.Client() as f:
    hours = await f.fetch_hours(span)
    # billing = await f.fetch_billing(span)
    # salesval = await f.fetch_salesvalue(span)
    # sales = await f.fetch_billing(span)

us = {user.guid: user.firstName for user in await f.users()}
us2 = {user.guid: user.workContract.hourCost.amount for user in await f.users()}
hours["name"] = hours.user.map(us)
hours["cost"] = hours.user.map(us2)

2023-07-06 08:10:39.073 | SUCCESS  | src.severa.base_client:get_with_retries:164 - HTTP/2 GET users [0]: Response (9) in 0.11s.
2023-07-06 08:10:39.204 | SUCCESS  | src.severa.base_client:get_with_retries:164 - HTTP/2 GET activities [0]: Response (100) in 0.12s.
2023-07-06 08:10:39.267 | SUCCESS  | src.severa.base_client:get_with_retries:164 - HTTP/2 GET salescases [0]: Response (8) in 0.19s.
2023-07-06 08:10:39.279 | SUCCESS  | src.severa.base_client:get_with_retries:164 - HTTP/2 GET users/af48336f-0979-b971-4927-9b09d53a3cef/workhours [0]: Response (100) in 0.19s.
2023-07-06 08:10:39.299 | SUCCESS  | src.severa.base_client:get_with_retries:164 - HTTP/2 GET users/eb8d73da-8c1c-7996-b085-332e777dff1a/workhours [0]: Response (100) in 0.22s.
2023-07-06 08:10:39.365 | SUCCESS  | src.severa.base_client:get_with_retries:164 - HTTP/2 GET users/0d100bc1-8376-7ac5-7bbb-055cdc20497d/workhours [0]: Response (100) in 0.16s.
2023-07-06 08:10:39.414 | SUCCESS  | src.severa.base_client:get_with_retr

In [15]:
u = src.severa.process.unravel(hours)
c = src.severa.process.cull_before(
    u,
    t.floor("day"),
    ["workhours", "saleswork"],  # absences?
    inclusive=False,
)
d = (
    c.groupby(["user", "id", "date", "productive", "project"], dropna=False)["value"]
    .sum()
    .reset_index()
)

d.id.unique()

C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(


<StringArray>
['absences', 'maximum', 'saleswork', 'workhours']
Length: 4, dtype: string

In [4]:
base = src.database.Base("kpi-dev-02", "billing")
billing_f = base.find({"forecast_date": arrow.utcnow().floor("day").datetime})

base = src.database.Base("kpi-dev-02", "hours")
hours_f = base.find({"forecast_date": arrow.utcnow().floor("day").datetime})

2023-07-04 20:49:38.562 | INFO     | src.database:find:90 - [billing] Query '{'forecast_date': datetime.datetime(2023, 7, 4, 0, 0, tzinfo=tzutc())}' resulted in 71 results in 1.69s.
2023-07-04 20:49:40.817 | INFO     | src.database:find:90 - [hours] Query '{'forecast_date': datetime.datetime(2023, 7, 4, 0, 0, tzinfo=tzutc())}' resulted in 226 results in 2.23s.


In [73]:
b = (
    src.severa.process.cull_before(
        src.severa.process.unravel(pd.concat([billing, billing_f], ignore_index=True)),
        arrow.utcnow().floor("day"),
        ["workhours", "billing"],
        inclusive=False,
    )
    .groupby(["user", "id", "date"])["value"]
    .sum()
    .reset_index()
)
b[b.value != 0]

cols = list(set(hours.columns) & set(hours_f.columns))
h1 = src.severa.process.sanitize_dates(
    hours, ["date", "start_date", "end_date", "forecast_date"]
)
h1 = src.severa.process.unravel(h1)

tot_hours1 = (
    y[y.id.isin(["workhours", "absences"])]
    .groupby(["user", "date"])["value"]
    .sum()
    .reset_index()
    .copy()
)
tot_hours1["id"] = "cost"
tot_hours1["value"] = cost.apply(lambda x: us2[x.user] * x.value, axis=1)


h2 = src.severa.process.sanitize_dates(
    hours_f, ["date", "start_date", "end_date", "forecast_date"]
)
h2 = src.severa.process.unravel(h2)

h3 = h1.merge(h2, on=cols, how="outer")

h = (
    src.severa.process.cull_before(
        h3,
        arrow.utcnow().floor("day"),
        ["workhours", "saleswork", "absences"],
        inclusive=False,
    )
    .groupby(["user", "id", "date"])["value"]
    .sum()
    .reset_index()
)

y = pd.concat([b[b.value != 0], h[h.value != 0]], ignore_index=True)
y = y[y.date.between(span.start.datetime, span.end.datetime)]

realized_cost = (
    y[(y.date <= arrow.utcnow().datetime) & (y.id.isin(["workhours", "absences"]))]
    .groupby(["user", "date"])["value"]
    .sum()
    .reset_index()
    .copy()
)


forecasted_cost = (
    y[(y.date > arrow.utcnow().datetime) & (y.id.isin(["maximum"]))]
    .groupby(["user", "date"])["value"]
    .sum()
    .reset_index()
    .copy()
)
print(forecasted_cost.value.mean())
print(forecasted_cost.sample(30))

cost = pd.concat([realized_cost, forecasted_cost], ignore_index=True)

cost["id"] = "cost"
cost["value"] = cost.apply(lambda x: us2[x.user] * x.value, axis=1)
cost.sample(50)

pd.concat([y, cost], ignore_index=True)

C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(
C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(
C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

7.162210338680927
                                      user                      date  value
59    0d100bc1-8376-7ac5-7bbb-055cdc20497d 2023-09-26 00:00:00+00:00    7.5
1059  2f365dcd-6ada-f6d8-8d73-4f393745e513 2024-09-25 00:00:00+00:00    7.5
1488  30c9d126-7cdc-53fc-3b05-7f98b25e3c98 2024-12-12 00:00:00+00:00    7.5
467   2441bfbf-78f2-0173-b0a1-8634ce167a0d 2023-11-13 00:00:00+00:00    7.5
3070  eb8d73da-8c1c-7996-b085-332e777dff1a 2023-10-23 00:00:00+00:00    7.5
1621  5cd93092-3385-3e89-003a-9fe6d03a424d 2024-01-02 00:00:00+00:00    1.5
1666  5cd93092-3385-3e89-003a-9fe6d03a424d 2024-03-05 00:00:00+00:00    1.5
511   2441bfbf-78f2-0173-b0a1-8634ce167a0d 2024-01-18 00:00:00+00:00    7.5
2214  78ddf0fe-9e39-5e5e-5096-a80409925de9 2024-11-11 00:00:00+00:00    7.5
1039  2f365dcd-6ada-f6d8-8d73-4f393745e513 2024-08-28 00:00:00+00:00    7.5
1023  2f365dcd-6ada-f6d8-8d73-4f393745e513 2024-08-06 00:00:00+00:00    7.5
325   0d100bc1-8376-7ac5-7bbb-055cdc20497d 2024-10-15 00:00:00+00:00  

,user,id,date,value
0,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-01 00:00:00+00:00,320.0
1,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-02 00:00:00+00:00,320.0
2,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-03 00:00:00+00:00,320.0
3,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-04 00:00:00+00:00,320.0
4,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-07 00:00:00+00:00,320.0
...,...,...,...,...
11036,eb8d73da-8c1c-7996-b085-332e777dff1a,cost,2024-12-17 00:00:00+00:00,363.75
11037,eb8d73da-8c1c-7996-b085-332e777dff1a,cost,2024-12-18 00:00:00+00:00,363.75
11038,eb8d73da-8c1c-7996-b085-332e777dff1a,cost,2024-12-19 00:00:00+00:00,363.75
11039,eb8d73da-8c1c-7996-b085-332e777dff1a,cost,2024-12-20 00:00:00+00:00,363.75


In [52]:
a = billing.groupby(["user", "id", "date"])["value"].sum().reset_index()
b = (
    src.severa.process.cull_before(
        src.severa.process.unravel(billing_f),
        arrow.utcnow().floor("day"),
        ["workhours", "billing"],
        inclusive=False,
    )
    .groupby(["user", "id", "date"])["value"]
    .sum()
    .reset_index()
)

c = pd.concat([a, b], ignore_index=True)
c  # 751
cc = c.groupby(["user", "id", "date"])["value"].sum().reset_index()

hours.groupby(["user", "id", "date"])["value"].sum().reset_index()
hours[pd.isna(hours.date)]

cols = list(set(hours.columns) & set(hours_f.columns))
h1 = src.severa.process.sanitize_dates(
    hours, ["date", "start_date", "end_date", "forecast_date"]
)
h2 = src.severa.process.sanitize_dates(
    hours_f, ["date", "start_date", "end_date", "forecast_date"]
)
h = h1.merge(h2, on=cols, how="outer")

x = (
    src.severa.process.cull_before(
        src.severa.process.unravel(h),
        arrow.utcnow().floor("day"),
        ["workhours", "saleswork", "abcenses", "billing"],
        inclusive=False,
    )
    .groupby(["user", "id", "date"])["value"]
    .sum()
    .reset_index()
)
print(len(x))
print(len(cc))

z = pd.concat([x, cc], ignore_index=True)
print(len(z))
print(len(z[z.value != 0]))
z[z.value != 0].sample(50)

C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(


33804
751
34555
23972


,user,id,date,value
17685,5cd93092-3385-3e89-003a-9fe6d03a424d,maximum,2024-04-11 00:00:00+00:00,1.5
32818,eb8d73da-8c1c-7996-b085-332e777dff1a,maximum,2026-01-02 00:00:00+00:00,7.5
16035,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,workhours,2024-09-20 00:00:00+00:00,1.015568
5616,2441bfbf-78f2-0173-b0a1-8634ce167a0d,maximum,2022-08-31 00:00:00+00:00,7.5
30767,eb8d73da-8c1c-7996-b085-332e777dff1a,maximum,2020-05-22 00:00:00+00:00,7.5
3740,0d100bc1-8376-7ac5-7bbb-055cdc20497d,workhours,2023-07-05 00:00:00+00:00,0.175562
6849,2441bfbf-78f2-0173-b0a1-8634ce167a0d,maximum,2026-01-15 00:00:00+00:00,7.5
13138,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,maximum,2024-06-06 00:00:00+00:00,7.5
7365,2441bfbf-78f2-0173-b0a1-8634ce167a0d,maximum,2027-06-15 00:00:00+00:00,7.5
20848,78ddf0fe-9e39-5e5e-5096-a80409925de9,maximum,2025-02-26 00:00:00+00:00,7.5


In [85]:
# print(hours.dtypes)

k = pd.DataFrame(
    data={
        "start_date": pd.NaT,
        "end_date": pd.NaT,
        "date": arrow.get("2023-01-01T12:00+0200").datetime,
        "value": [10.0, 8.0, 13.0],
    },
    index=[0, 1, 2],
)

cols = ["start_date", "end_date", "date"]
subset = k.loc[:, cols]
for c in cols:
    k[c] = pd.to_datetime(k.loc[:, c], utc=True)

k.date.dt.tz is datetime.timezone.utc

True

In [81]:
d = src.severa.process.unravel(hours)
d2 = src.severa.process.cull_before(d, arrow.utcnow().floor("day"), ["workhours"])
d3 = d2[("2023-01-01" <= d2.date) & (d2.date < "2023-10-01")]
# d3["total"] = d3.absences + d3.saleswork + d3.workhours
d3["total_cost"] = d3.value * d3.cost
# pd.pivot_table(d3, values=["value", "total_cost"], columns=["id"], index=[d3.date.dt.month, "name"], aggfunc=sum, fill_value=0).reset_index()

billing["name"] = billing.user.map(us)
b1 = src.severa.process.unravel(billing)
b1.groupby([b1.date.dt.month])["value"].sum().reset_index()

d4 = d3[d3.id != "maximum"]
d4.groupby([d4.date.dt.month])["total_cost"].sum().reset_index()

C:\Users\vireima\tie-dashboard\src\severa\process.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(
C:\Users\vireima\AppData\Local\Temp\ipykernel_20900\3774696409.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d3["total_cost"] = d3.value * d3.cost
C:\Users\vireima\tie-dashboard\src\severa\process.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,date,total_cost
0,1,65255.955645
1,2,60758.58871
2,3,69948.439516
3,4,55455.536342
4,5,65170.208535
5,6,45745.082325
6,7,32600.293436
7,8,35226.940083
8,9,30624.146885


In [63]:
hours[hours.productive.isna()].id.unique()

<StringArray>
['maximum', 'absences']
Length: 2, dtype: string

In [27]:
us = {user.guid: user.firstName for user in await f.users()}
d = src.severa.process.unravel(
    hours
)  # [hours.id.isin(["workhours", "saleshours", "absences"])])
# d = src.severa.process.unravel(hours[hours.id.isin(["maximum"])], arrow.get("2023-03-01").datetime, arrow.get("2023-09-01").datetime)
d["name"] = d.user.map(us)
x = src.severa.process.cull_before(d, arrow.utcnow().floor("day"), ["workhours"])


d2 = x[("2023-06-01" <= x.date) & (x.date < "2023-10-01")]


pd.pivot_table(
    d2,
    values=["value"],
    columns=["id"],
    index=[d2.date.dt.month, "name"],
    aggfunc=sum,
    fill_value=0,
).reset_index()
# hours[hours.id == "maximum"]

C:\Users\vireima\tie-dashboard\src\severa\process.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(


date       name     value                                 
id                      absences  maximum    saleswork   workhours
0          6    Johanna      52.5     157.5   1.991613   42.294511
1          6       Joni      15.0     157.5      7.875  110.348308
2          6       Lari      15.0     157.5        0.0    141.8125
3          6  Margarita       2.0     157.5    2.67052  143.217196
4          6      Miika       0.0     157.5        0.0   14.136094
5          6       Panu       0.0      31.5        0.0         0.0
6          6       Topi      15.0     157.5        0.0   97.393574
7          6      Ville       0.0     157.5        0.0   160.68209
8          6  Vladimirs       7.5     157.5        0.0  103.772486
9          7    Johanna     150.0     157.5   1.991613    3.294101
10         7       Joni       0.0     157.5      7.875   56.144925
11         7       Lari     150.0     157.5        0.0         3.5
12         7  Margarita      37.5     157.5    2.67052   38.310908
13         7      Miika       0.0     157.5        0.0   10.193483
14         7       Panu       0.0      31.5        0.0         0.0
15         7       Topi       0.0     157.5        0.0   47.579986
16         7      Ville      75.0     157.5        0.0   18.985294
17         7  Vladimirs       0.0     157.5        0.0   67.896899
18         8    Johanna       0.0     172.5    2.18129   75.764315
19         8       Joni       0.0     172.5     15.325   73.492061
20         8       Lari       0.0     172.5  13.176601  106.382353
21         8  Margarita       0.0     172.5   2.924855    54.09331
22         8      Miika       0.0     172.5        0.0   13.164291
23         8       Panu       0.0      34.5        0.0         0.0
24         8       Topi       0.0     172.5        0.0  132.726064
25         8      Ville       7.5     172.5        0.0  122.102941
26         8  Vladimirs       0.0     172.5        0.0    87.85971
27         9    Johanna       0.0     157.5   8.908843   68.926114
28         9       Joni       0.0     157.5  13.992391   56.144925
29         9       Lari       0.0     157.5   12.03081   107.49443
30         9  Margarita       0.0     157.5    2.67052   26.292365
31         9      Miika       0.0     157.5        0.0   12.193483
32         9       Panu       0.0      31.5        0.0         0.0
33         9       Topi       0.0     157.5        0.0  102.866091
34         9      Ville       7.5     157.5        0.0  111.416667
35         9  Vladimirs       0.0     157.5        0.0   89.319735

In [139]:
hours.loc[hours.date.isna() & (hours.id == "maximum"), "user"]

d = pd.DataFrame(
    {"date": pd.date_range("2023-05-01", "2023-05-30", freq="D", tz="utc")}
)
d["mask"] = d.date.map(Finland().is_working_day).map(int)
d

x = (
    hours.loc[hours.date.isna() & (hours.id == "maximum"), :]
    .drop("date", axis=1)
    .merge(d, how="cross")
)
x["value"] *= x["mask"]

In [157]:
x = src.severa.process.unravel_maximum(hours)
hours[hours.id == "maximum"]

,user,value,id,date,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date
0,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-15 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
1,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-16 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
2,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-17 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
3,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-18 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
4,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-19 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
5,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-20 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
6,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-21 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
7,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-22 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
8,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-23 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00


In [178]:
def unravel_maximum(data: pd.DataFrame) -> pd.DataFrame:
    min_date = data.loc[:, ["date", "start_date"]].min().min()
    max_date = data.loc[:, ["date", "end_date"]].max().max()
    print(min_date)
    print(max_date)
    dt_index = pd.date_range(min_date, max_date, freq="D", tz="utc")
    dates = pd.DataFrame({"date": dt_index})

    dates["workday_mask"] = dates.date.map(Finland().is_working_day).map(int)
    unraveled_subset = (
        data.loc[data.id == "maximum", :].drop("date", axis=1).merge(dates, how="cross")
    )
    unraveled_subset["value"] *= unraveled_subset["workday_mask"]

    # data.merge(unraveled_subset, on=["id", "user"])
    # data.loc[data.date.isna() & (data.id == "maximum"), :] = masking.drop(
    #    "workday_mask", axis=1
    # )

    return unraveled_subset


hours[hours.id == "maximum"]
hours[hours.id != "maximum"]

pd.concat(
    [unravel_maximum(hours).drop("workday_mask", axis=1), hours[hours.id != "maximum"]],
    ignore_index=True,
)

2020-03-15 00:00:00+00:00
2027-11-30 00:00:00+00:00


,user,value,id,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date,date
0,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-15 00:00:00+00:00
1,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-16 00:00:00+00:00
2,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-17 00:00:00+00:00
3,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-18 00:00:00+00:00
4,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-19 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25915,0d100bc1-8376-7ac5-7bbb-055cdc20497d,73.5,saleswork,<NA>,f62568a5-36ce-5dc3-490d-a3a24031dba1,<NA>,<NA>,<NA>,2022-11-03 00:00:00+00:00,2025-12-01 00:00:00+00:00,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,2023-06-16 00:00:00+00:00,NaT
25916,2441bfbf-78f2-0173-b0a1-8634ce167a0d,22.0,saleswork,<NA>,e98dd28a-27be-a646-3141-bef922976214,<NA>,<NA>,<NA>,2023-04-25 00:00:00+00:00,2023-12-30 00:00:00+00:00,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-06-16 00:00:00+00:00,NaT
25917,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,12.6,saleswork,<NA>,1913890d-2f2f-c164-3e71-e532a6c96ebe,<NA>,<NA>,<NA>,2023-08-01 00:00:00+00:00,2024-10-31 00:00:00+00:00,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-16 00:00:00+00:00,NaT
25918,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,1.6,saleswork,<NA>,1913890d-2f2f-c164-3e71-e532a6c96ebe,<NA>,<NA>,<NA>,2025-05-01 00:00:00+00:00,2025-06-30 00:00:00+00:00,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-16 00:00:00+00:00,NaT


In [198]:
import random

dat = pd.DataFrame(
    {
        "id": list("AAABBB"),
        "value": [10, 8, 3, 7, 1, 1],
        "start_date": pd.Timestamp("2023-06-01"),
        "end_date": pd.Timestamp("2023-06-08"),
    }
)
dat.loc[1, "start_date"] = pd.Timestamp("2023-05-28")
dat.loc[2, "end_date"] = pd.Timestamp("2023-06-9")

dat["date"] = dat.apply(
    lambda x: pd.date_range(start=x["start_date"], end=x["end_date"]), axis=1
)
dat["_num_workdays"] = dat.apply(
    lambda x: Finland().get_working_days_delta(
        x["start_date"].date(), x["end_date"].date()
    ),
    axis=1,
)

dat = dat.explode("date").reset_index(drop=True)
dat["_is_workday"] = dat.date.map(Finland().is_working_day).map(int)
dat

hours[hours.date.isna()].sample(20)

,user,value,id,date,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date
516,2f365dcd-6ada-f6d8-8d73-4f393745e513,4.0,workhours,NaT,<NA>,efd040a7-b946-7f32-cdc7-5254d4707403,<NA>,False,e4a802ac-54f4-d3f2-6b0c-229f94118e7d,2023-06-01 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
545,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,25.0,workhours,NaT,<NA>,f1a3d420-e970-74be-7668-ab3104c00551,<NA>,False,1915e2b0-c434-0322-9175-f9454a283e70,2023-06-26 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
563,2441bfbf-78f2-0173-b0a1-8634ce167a0d,67.5,workhours,NaT,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,<NA>,False,94b5fc97-f7cc-657a-a4db-b5803afd737f,2023-02-28 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
546,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,91.0,workhours,NaT,<NA>,2363278a-275f-ce2c-7a3b-5beea945c845,<NA>,False,67e159dc-1266-5c1c-51fa-228c95a1ea73,2022-10-01 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
538,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,8.0,workhours,NaT,<NA>,bf48e17f-7503-7ec5-87a8-25ad5cebd210,<NA>,False,079766cf-f3fd-80df-b9dd-e9bfc9a012e2,2023-06-01 00:00:00+00:00,2023-06-25 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
555,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,27.0,workhours,NaT,<NA>,7dff98a8-da80-1006-4bb1-2bda3507c7d4,<NA>,False,fb38fbce-53da-62a6-57e2-5ff874ff5950,2023-05-12 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
530,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,10.0,workhours,NaT,<NA>,bc7710f2-37a0-d14b-fcd4-a9e4fe8a95dd,<NA>,False,39aad51a-e04d-7880-daa1-fef750b382d2,2023-06-01 00:00:00+00:00,2023-06-25 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
490,eb8d73da-8c1c-7996-b085-332e777dff1a,2.0,workhours,NaT,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,<NA>,False,94b5fc97-f7cc-657a-a4db-b5803afd737f,2023-02-28 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
514,2f365dcd-6ada-f6d8-8d73-4f393745e513,31.0,workhours,NaT,<NA>,c1e8f641-cadc-06ac-fa6a-334e8afaf814,<NA>,True,2b8f4b28-c55c-2326-66fd-6ad56748c059,2023-03-02 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
565,2441bfbf-78f2-0173-b0a1-8634ce167a0d,44.0,workhours,NaT,<NA>,dece01c7-5324-c4b4-2bab-497009174f92,<NA>,False,ff9a747b-a7de-1d49-4cca-adb8ffa19180,2023-04-07 00:00:00+00:00,2023-07-16 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00


In [301]:
y = src.severa.process.unravel(hours)

C:\Users\vireima\tie-dashboard\src\severa\process.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,"_num_workdays"] = data.loc[:,:].apply(


In [302]:
y

,user,value,id,date,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date,xxx
0,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-15 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
1,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-16 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
2,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-17 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
3,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-18 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
4,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-19 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40840,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2.5,workhours,2023-06-15 00:00:00+00:00,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,True,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
40841,2441bfbf-78f2-0173-b0a1-8634ce167a0d,0.5,workhours,2023-06-15 00:00:00+00:00,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,True,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
40842,2441bfbf-78f2-0173-b0a1-8634ce167a0d,1.0,workhours,2023-06-15 00:00:00+00:00,<NA>,bad86762-4521-1d09-a992-473607c75c48,False,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
40843,2441bfbf-78f2-0173-b0a1-8634ce167a0d,1.0,workhours,2023-06-15 00:00:00+00:00,<NA>,bad86762-4521-1d09-a992-473607c75c48,False,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
